In [1]:
#coding:utf-8

# ライブラリのダウンロード
import requests
from bs4 import BeautifulSoup
# import urllib
import urllib.request
import sys
import zipfile
import csv
import os
import pyprind
from pathlib import Path
import shutil
import pandas as pd
import re

In [2]:
# 変数
csvUrl= "https://www.aozora.gr.jp/index_pages/list_person_all_extended_utf8.zip"
csvFileName = "list_person_all_extended_utf8"
autherName=["芥川竜之介","太宰治","夏目漱石","宮沢賢治"]
autherID = ["000879","000035","000148","000081"]
dataPath = "aozora"
trainPath = "train"
testPath = "test"

In [33]:
# 青空文庫の作者情報csvをダウンロード(Zip)
urllib.request.urlretrieve(csvUrl,csvFileName+".zip")

('list_person_all_extended_utf8.zip',
 <http.client.HTTPMessage at 0x7f5cf02e9278>)

In [34]:
# Zipを展開する
with zipfile.ZipFile(csvFileName+".zip") as existing_zip:
    existing_zip.extractall('./')

In [3]:
# csvを読み込む
csvReader = ""
with open("./"+csvFileName+".csv",encoding='utf-8') as f:
    reader = csv.reader(f)
    csvReader = [row for row in reader]

In [4]:
# 保存するフォルダーの作成
if not os.path.exists(dataPath):
    os.mkdir(dataPath)
for auther in autherName:
    if not os.path.exists(dataPath+"/"+auther):
        os.mkdir(dataPath+"/"+auther)

In [5]:
# 青空文庫から作品をまとめてダウンロードするので時間がかかります！
humanIdOrder = csvReader[0].index("人物ID")
textFileOrder = csvReader[0].index("XHTML/HTMLファイルURL")
workNameOrder = csvReader[0].index("作品名")
roleFlag = csvReader[0].index("役割フラグ")
pbar = pyprind.ProgBar(len(csvReader))

# 青空文庫のファイルを保存する
def saveAozoraFile(auther,work,url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'lxml')
    elem = soup.find(class_='main_text')
    if elem==None:
        return
    rp_all = elem.find_all('rp')
    for rp in rp_all:
        rp.extract()
    rt_all = elem.find_all('rt')
    for rt in rt_all:
        rt.extract()

    text = elem.get_text()
    with open(dataPath+"/"+auther+"/"+work+".txt","w",encoding='utf-8')as f:
        f.write(text)

for row in csvReader:
    pbar.update()
    if autherID.count(row[humanIdOrder])!=0 and row[roleFlag]=="著者" and re.match("https?://www.aozora.gr.jp/cards/",row[textFileOrder]) and "_" in row[textFileOrder]:
        selectAuther = autherID.index(row[humanIdOrder])
        print(autherName[selectAuther],row[workNameOrder],row[textFileOrder])
        if not os.path.exists(dataPath+"/"+autherName[selectAuther]+"/"+row[workNameOrder]+".txt"):
            saveAozoraFile(autherName[selectAuther],row[workNameOrder],row[textFileOrder])

芥川竜之介 愛読書の印象 https://www.aozora.gr.jp/cards/000879/files/4872_21839.html
芥川竜之介 秋 https://www.aozora.gr.jp/cards/000879/files/16_14570.html
芥川竜之介 芥川竜之介歌集 https://www.aozora.gr.jp/cards/000879/files/178_14584.html
芥川竜之介 アグニの神 https://www.aozora.gr.jp/cards/000879/files/15_14583.html
芥川竜之介 アグニの神 https://www.aozora.gr.jp/cards/000879/files/43014_17430.html
芥川竜之介 悪魔 https://www.aozora.gr.jp/cards/000879/files/3804_27277.html
芥川竜之介 浅草公園 https://www.aozora.gr.jp/cards/000879/files/21_15157.html
芥川竜之介 兄貴のような心持 https://www.aozora.gr.jp/cards/000879/files/43361_17877.html
芥川竜之介 あの頃の自分の事 https://www.aozora.gr.jp/cards/000879/files/17_14601.html
芥川竜之介 あばばばば https://www.aozora.gr.jp/cards/000879/files/14_14602.html
芥川竜之介 鴉片 https://www.aozora.gr.jp/cards/000879/files/1138_15285.html
芥川竜之介 或阿呆の一生 https://www.aozora.gr.jp/cards/000879/files/19_14618.html
芥川竜之介 或敵打の話 https://www.aozora.gr.jp/cards/000879/files/73_15158.html
芥川竜之介 或旧友へ送る手記 https://www.aozora.gr.jp/cards/000879/files/20_14619.html
芥川竜之介

0% [##############                ] 100% | ETA: 00:00:00

芥川竜之介 路上 https://www.aozora.gr.jp/cards/000879/files/132_15264.html
芥川竜之介 LOS CAPRICHOS https://www.aozora.gr.jp/cards/000879/files/94_15098.html
芥川竜之介 露訳短篇集の序 https://www.aozora.gr.jp/cards/000879/files/24455_11250.html
芥川竜之介 わが散文詩 https://www.aozora.gr.jp/cards/000879/files/3812_27356.html
芥川竜之介 わが俳諧修業 https://www.aozora.gr.jp/cards/000879/files/3771_26611.html
芥川竜之介 わが家の古玩 https://www.aozora.gr.jp/cards/000879/files/3798_27357.html
芥川竜之介 忘れられぬ印象 https://www.aozora.gr.jp/cards/000879/files/3806_27358.html
芥川竜之介 私の好きなロマンス中の女性 https://www.aozora.gr.jp/cards/000879/files/4871_21840.html
太宰治 ア、秋 https://www.aozora.gr.jp/cards/000035/files/236_19996.html
太宰治 I can speak https://www.aozora.gr.jp/cards/000035/files/1572_19910.html
太宰治 愛と美について https://www.aozora.gr.jp/cards/000035/files/1578_44923.html
太宰治 青森 https://www.aozora.gr.jp/cards/000035/files/4357_7911.html
太宰治 青森 https://www.aozora.gr.jp/cards/000035/files/46597_24917.html
太宰治 朝 https://www.aozora.gr.jp/cards/000035/files/1562_148

0% [########################      ] 100% | ETA: 00:00:00

https://www.aozora.gr.jp/cards/000035/files/315_20041.html
太宰治 『ろまん灯籠』序 https://www.aozora.gr.jp/cards/000035/files/54178_46738.html
太宰治 わが愛好する言葉 https://www.aozora.gr.jp/cards/000035/files/42374_34756.html
太宰治 わが半生を語る https://www.aozora.gr.jp/cards/000035/files/1601_18118.html
太宰治 私の著作集 https://www.aozora.gr.jp/cards/000035/files/52292_45480.html
太宰治 私の著作集 https://www.aozora.gr.jp/cards/000035/files/45673_21431.html
太宰治 渡り鳥 https://www.aozora.gr.jp/cards/000035/files/2295_15090.html
夏目漱石 イズムの功過 https://www.aozora.gr.jp/cards/000148/files/2314_13564.html
夏目漱石 一夜 https://www.aozora.gr.jp/cards/000148/files/1086_14935.html
夏目漱石 永日小品 https://www.aozora.gr.jp/cards/000148/files/758_14936.html
夏目漱石 岡本一平著並画『探訪画趣』序 https://www.aozora.gr.jp/cards/000148/files/2669_6501.html
夏目漱石 思い出す事など https://www.aozora.gr.jp/cards/000148/files/792_14937.html
夏目漱石 カーライル博物館 https://www.aozora.gr.jp/cards/000148/files/1046_14938.html
夏目漱石 薤露行 https://www.aozora.gr.jp/cards/000148/files/769_14939.html
夏目漱石 学者と名

0% [##############################] 100% | ETA: 00:00:00

宮沢賢治 釜石よりの帰り https://www.aozora.gr.jp/cards/000081/files/53409_43256.html
宮沢賢治 烏の北斗七星 https://www.aozora.gr.jp/cards/000081/files/1928_17905.html
宮沢賢治 烏の北斗七星 https://www.aozora.gr.jp/cards/000081/files/43755_17660.html
宮沢賢治 雁の童子 https://www.aozora.gr.jp/cards/000081/files/461_27798.html
宮沢賢治 革トランク https://www.aozora.gr.jp/cards/000081/files/4426_29930.html
宮沢賢治 看痾 https://www.aozora.gr.jp/cards/000081/files/53411_43257.html
宮沢賢治 〔甘藍の球は弾けて〕 https://www.aozora.gr.jp/cards/000081/files/53374_43258.html
宮沢賢治 黄いろのトマト https://www.aozora.gr.jp/cards/000081/files/1919_18095.html
宮沢賢治 機会 https://www.aozora.gr.jp/cards/000081/files/53392_43259.html
宮沢賢治 饑餓陣営 https://www.aozora.gr.jp/cards/000081/files/1921_17653.html
宮沢賢治 疑獄元兇 https://www.aozora.gr.jp/cards/000081/files/48221_32492.html
宮沢賢治 気のいい火山弾 https://www.aozora.gr.jp/cards/000081/files/46606_30108.html
宮沢賢治 気のいい火山弾 https://www.aozora.gr.jp/cards/000081/files/4440_7303.html
宮沢賢治 饗宴 https://www.aozora.gr.jp/cards/000081/files/53361_43260.ht


Total time elapsed: 00:00:01


In [15]:
# 保存したファイルをtainとtestにわけるフォルダーの作成
# if not os.path.exists(trainPath):
#     os.mkdir(trainPath)
# if not os.path.exists(testPath):
#     os.mkdir(testPath)
# for auther in autherName:
#     if not os.path.exists(trainPath+"/"+auther):
#         os.mkdir(trainPath+"/"+auther)
# for auther in autherName:
#     if not os.path.exists(testPath+"/"+auther):
#         os.mkdir(testPath+"/"+auther)

In [46]:
# # trainとtestに分ける
# for auther in autherName:
#     pList=Path(dataPath+"/"+auther)
#     aozoraList = list(p.glob('*.txt'))
#     if len(aozoraList)<130:
#         print('Error: '+auther+' のファイルが足りません。作者を変更してください。', file=sys.stderr)
#         sys.exit(1)
        
#     trainList = aozoraList[0:99]
#     testList = aozoraList[100:129]
#     for file in trainList:
#         fileName=Path(file).name
#         shutil.copyfile(str(file),trainPath+"/"+auther+"/"+fileName)
#     for file in testList:
#         fileName=Path(file).name
#         shutil.copyfile(str(file),testPath+"/"+auther+"/"+fileName)

In [9]:
# トレーニングデータを読み込む
df = pd.DataFrame()
# testとtrainのフォルダを探索する
for s in ([dataPath]):
    #  posとnegのフォルダを探索する
    for l,i in zip(autherName,range(len(autherName))):
        # フォルダ名を結合する
        path = os.path.join("./", s, l)
        print(l,len(os.listdir(path)))
        # ファイルを開いてpandasのDataFrameに追加する
        for j,file in enumerate(sorted(os.listdir(path))):
            if j>=100:
                break
            with open(os.path.join(path, file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, i]],ignore_index=True)
df.columns = ['ドキュメント', '作者ID']


芥川竜之介 355
b
太宰治 252
b
夏目漱石 101
b
宮沢賢治 245
b


In [10]:
# シャッフルする
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

In [11]:
# 読み込めているかチェック
print(df.shape)
df.head(3)

(400, 2)


,ドキュメント,作者ID
132,\nはしがき\n\n もの思う葦という題名にて、日本浪曼派の機関雑誌におよそ一箇年ほどつづけ...,1
309,\nこのみちの醸すがごとく\n粟葉などひかりいでしは\nひがしなる山彙の上に\n黄なる月いざ...,3
341,\n洪積の台のはてなる\n一ひらの赤き粘土地\n\n桐の群白くひかれど\n枝しげくたけ低けれ...,3


In [12]:
# Mecabの環境が必要です
# Mecabが動くかのテスト
import MeCab
mecab = MeCab.Tagger("-Owakati")
text = mecab.parse ("今日の中野の天気は晴れです。")
print(text)

今日 の 中野 の 天気 は 晴れ です 。 



In [13]:
# Mecabを使って、日本語を形態素解析する関数を定義
# Mecabの環境が必要です
# http://taku910.github.io/mecab/#download
import MeCab

def preprocessor(text):
    mecab = MeCab.Tagger("-Owakati")
    text=re.sub("\\n","",text)
    return mecab.parse(text)

In [14]:
# すべてのドキュメントに関数を使う
df['ドキュメント'] = df['ドキュメント'].apply(preprocessor)

In [15]:
df.head(10)

,ドキュメント,作者ID
132,はし が き もの 思う 葦 という 題名 にて 、 日本 浪 曼派 の 機関 雑誌 に...,1
309,この みち の 醸す が ごとく 粟 葉 など ひかり い でし は ひがし なる 山 彙 ...,3
341,洪 積 の 台 の はて なる 一 ひ ら の 赤き 粘土 地 桐 の 群 白く ひか れ ...,3
196,（ わが 陋屋 に は 、 六 坪 ほど の 庭 が ある の だ 。 愚妻 は 、 ここ ...,1
246,さっき から 松原 を 通っ てる ん だ が 、 松原 と 云う もの は 絵 で 見...,2
60,僕 は 中学 五 年生 の 時 に 、 ドオデエ の 「 サッフォ 」 という 小説 の...,0
155,芸術 家 という もの は 、 つくづく 困っ た 種族 で ある 。 鳥 籠 一つ を...,1
261,私 は この 大阪 で 講演 を やる の は 初めて で あり ます 。 また こうい...,2
141,一 人 の 遊蕩 の 子 を 描写 し て 在る ゆえ を以て 、 その 小説 を 、 ...,1
214,「 煤煙 」 が 朝日新聞 に 出 て 有名 に な つて から 後 間もなく の 話 で ...,2


In [16]:
# 日本語のstopを定義
# 出典：https://qiita.com/chamao/items/7edaba62b120a660657e
def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(file_path):
    stop_words = []
    for w in open(path, "r",encoding='utf-8'):
        w = w.replace('\n','')
        if len(w) > 0:
            stop_words.append(w)
    return stop_words    

path = "stop_words.txt"
download_stopwords(path)
stop = create_stopwords(path)

File already exists.


In [18]:
# trainとtestデータに分ける
X_train = df.loc[:300, 'ドキュメント'].values
y_train = df.loc[:300, '作者ID'].values
X_test = df.loc[300:, 'ドキュメント'].values
y_test = df.loc[300:, '作者ID'].values

In [19]:
def tokenizer(text):
    return text.split()
def tokenizer_porter(text):
    return text.split()

In [21]:
# 5分割交差検定でロジスティック回帰モデルの最適なパラメータ集合を求める

# 日本語のtfidfの参照
# https://analytics-note.xyz/machine-learning/bow-tfidf-one-character/

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None,
                       token_pattern='(?u)\\b\\w+\\b',
                       max_features=5000)

# 1つ目はデフォルト値
# 2つ目は生の出現頻度でトレーニング
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer],
#                               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer],
#                               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

# pipelineに入れる
lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

# 注意！！gs_lr_tfidfのn_jobs=-1でマルチプロセス処理を最大で実行。
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [22]:
# この処理は30分-60分かかる恐れがあります!!!!!!!!!!
# トレーニング開始
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   11.1s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=5000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
 ...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい...se_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=True, return_tr

In [23]:
# ベストなパラメーター集合とそのスコア
print('Best parameter set: %s ' % gs_lr_tfidf.best_params_)
print('CV Accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7ff247fe5ae8>} 
CV Accuracy: 0.871


In [24]:
# 90%程度の正解率で肯定的か否定的かを判定できる
clf = gs_lr_tfidf.best_estimator_
print('Test Accuracy: %.3f' % clf.score(X_test, y_test))

Test Accuracy: 0.839
